## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head()
print(data.describe())





           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  
count      21525.000000  21525.000000  1.935100e+04  
mean           0.972544      0.080883  1.674223e+05  
std            1.420324      0.272661  1.029716e+05  
min            0.000000      0.000000  2.066726e+04  
25%            0.000000      0.000000  1.030532e+05  
50%            0.000000      0.000000  1.450179e+05  
75%            1.000000    

# Вывод


В файле много моментов, которые надо исправить - трудовой стаж отрицательный, уровень образования написан разным регистром.В 2174 строках из 21525 пропущенно значение трудового стажа и ежемесячного дохода. Необычное значение количества детей -1 и 20.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [22]:
print(data.isnull().sum())
print(data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()) # проверяем в одних и тех же строках 
#ли находятся пропуски в столбцах "стаж" и "доход"
print(min(data['days_employed'])) #вычисляем минимальные и максимальные значения
print(max(data['days_employed']))
print(min(data['total_income']))
print(max(data['total_income']))
days_med = data['days_employed'].median() #вычисляем медианы
data['days_employed'] = data['days_employed'].fillna(value = days_med) #приминяем метод замены нулевых значений на медианные.
inc_med = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value = inc_med)
print(data.isnull().sum())


data_pivo = data.pivot_table(index=['income_type'], values='total_income') #сводная таблица по типу занятости и уровню дохода.
print(data_pivo) #из таблицы мы видим,что значения  дохода для разных профессий близки к медианным. Выделяются только "в декрете" 
# и студент. Пропуски в этих строках лучше заменять медианными значениями по профессии.





children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
lem_purpose              0
purpose_category         0
children_category        0
family_category          0
total_income_category    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 17 columns):
children                 0 non-null int64
days_employed            0 non-null int64
dob_years                0 non-null float64
education                0 non-null object
education_id             0 non-null int64
family_status            0 non-null object
family_status_id         0 non-null int64
gender                   0 non-null object
income_type              0 non-null object
debt                     0 non-null int64
total_inc

# Вывод

В столбцах "трудовой стаж" и "ежемесячный обнаружил пропуски. Убедился, что они находятся в одних и тех же строках. Посмотрел максимальное и минимальное значение этих столбцов. Значения доходов разнятся в 10 раз - поэтому решил заменить пропуски в этом столбце медианным значением. В столбце "стаж" имеются отрицательные значения, а если взять эти значения по модулю, то максимальное и среднее значение будут слишком большими - заменим пропуски на медианные значения.

### Замена типа данных

In [24]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data[data['total_income'] > 0].count()[0]) #все значения столбца доход - положительные
data['days_employed'] = abs(data['days_employed']) # берем модуль значений столбца стаж
print(data.head())
print(data['education'].unique()) #смотрим, как записаны уровни образования
data['education'] = data['education'].str.lower()
print(data['education'].unique()) #проверяем как отработал метод
print(min(data['dob_years']))
print(max(data['dob_years']))
print(data['income_type'].unique()) #смотрим как записан тип занятости
print(data[data['dob_years'] == 0].count()[0]) #cмотрим сколько людей с возрастом 0 лет
years_avg = data['dob_years'].mean() #высчитаем средний возраст
data['dob_years'] = data['dob_years'].replace(0, years_avg) #замени нулевой возраст на средний
print(data[data['children'] == -1].count()[0]) #смотрим сколько людей сколичеством детей -1
data['children'] = data['children'].replace(-1, 0) #заменим значение -1 на 0 в толбце дети
print(data[data['children'] == 20].count()[0]) #смотрим сколько людей сколичеством детей 20.
# 76 людей имеют по 20 детей. это странные неправдоподобные значения. Предположу, что это ошибка в сборе данных и заменю значение 20 на 2
data['children'] = data['children'].replace(20, 2) 


21454
   children  days_employed  dob_years education  education_id  \
0         1           8437       42.0    высшее             0   
1         1           4024       36.0   среднее             1   
2         0           5623       33.0   среднее             1   
3         3           4124       32.0   среднее             1   
4         0         340266       53.0   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose                                   lem_purpose  \
0               покупка жилья              

### Вывод

Перевел данные столбцов "трудовой стаж" и "ежемесячный доход в целые величины методом astype. Также взял модуль значений столбца "трудовой стаж",т.к. там были отрицательные значения, видимо из-за ошибки сбора данных. Привел к нижнему регистру значения столбца "образование" методом str.lower. В столбце "возраст" обнаружил нулевое значение, заменил его на среднее по столбцы "возраст". Значения -1 в столбце "дети" заменил на 0, предположив, что отрицательное значение появилось по ошибке.

### Обработка дубликатов

In [5]:
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop= True)
print(data.duplicated().sum())

71
0


# Вывод


Было выявленно 54 дубликата. Удалил методом drop_duplicates(). Причиной появления дубликатов могла стать ошибка на этапе сбора и формирования данных.

### Лемматизация

In [6]:
from pymystem3 import Mystem
m = Mystem()
purpose_unique = data['purpose'].value_counts().index.to_list()
print(purpose_unique) #определяем уникальные значения столбца "цель кредита"
for i in range(len(data)): #лемматизируем список "цели кредита" сохраняем его в новый столбец "lem_purpose"
    lemmas =m.lemmatize(data['purpose'][i])
    data.loc[i,'lem_purpose'] = str(lemmas) #изменяем тип объекта lemmas на string
print(data.head())
from collections import Counter
print(Counter(data['lem_purpose'])) #считаем количество лемм
purpose_category = ['свадьба','недвижимость','жилье','автомобиль','образование'] # наиболее частовстречаемые категории
def categorize_purpose(text): #пишем функцию, которая возвращает категорю цели на основе лемматезированного списка
   
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in purpose_category:
            if category in lemma:
                return category
data['purpose_category'] = data['purpose'].apply(categorize_purpose) #добавляем столбец "категория цели кредита"
print(data.head(10))

['свадьба', 'на проведение свадьбы', 'сыграть свадьбу', 'операции с недвижимостью', 'покупка коммерческой недвижимости', 'операции с жильем', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью', 'покупка жилья', 'жилье', 'покупка жилья для семьи', 'строительство собственной недвижимости', 'недвижимость', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости', 'на покупку своего автомобиля', 'заняться высшим образованием', 'автомобиль', 'сделка с подержанным автомобилем', 'свой автомобиль', 'на покупку подержанного автомобиля', 'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'дополнительное образование', 'сделка с автомобилем', 'высшее образование', 'образование', 'получение дополнительного образования', 'получение образования', 'профильное образование', 'получение высшего образования', 'заняться образованием']
   chi

### Вывод

Лемматизировали столбец целей кредита и подсчитали количество упоминаний счетчиком Counter. Написали функцию, котрая определяет категорию. Создали стобец "категория цели кредита"

### Категоризация данных

In [27]:
def categorize_children(count): #функция для определения категории семьи по количеству детей
    categories = { 0: 'бездетная', 1: 'однодетная', 2: 'двудетная' }

    return categories.get(count, 'многодетная')
data['children_category'] = data['children'].apply(categorize_children) #добавляем столбец с категорией семьи по кол-ву детей


def categorize_family_status(value): #функция для определения категории семьи по family_status

    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'
data['family_category'] = data['family_status'].apply(categorize_family_status) #добавляем столбец с категорией семьи по браку
#по уровню дохода разбил данные на три группы. Средним уровнем буду считать медианное значение. 
def categorize_total_income (value): #функция для определения категории уровня дохода.
    if value < 50000: #минимальное значение у нас 20667 а медианное 145017. цифру 50000 взял случайно,чтобы сделать категорию "низкий"
        return 'низкий'
    elif value <= inc_med: #медианное значение уровндя дохода
        return 'средний'
    elif value <=2265604: #максимальное значение уровня дохода
        return 'выше среднего'
    
    return 'высокий'
data['total_income_category'] = data['total_income'].apply(categorize_total_income) #добавляем столбец с категорией дохода
print(data.head(20))
print(min(data['total_income']))
print(max(data['total_income']))
print(inc_med)

    children  days_employed  dob_years            education  education_id  \
0          1           8437       42.0               высшее             0   
1          1           4024       36.0              среднее             1   
2          0           5623       33.0              среднее             1   
3          3           4124       32.0              среднее             1   
4          0         340266       53.0              среднее             1   
5          0            926       27.0               высшее             0   
6          0           2879       43.0               высшее             0   
7          0            152       50.0              среднее             1   
8          2           6929       35.0               высшее             0   
9          0           2188       41.0              среднее             1   
10         2           4171       36.0               высшее             0   
11         0            792       40.0              среднее             1   

### Вывод

Создал столбцы с необходимыми категориями, для ответов на поставленные в задаче вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
data_pivot = data.pivot_table(index=['children_category'], values='debt') #сводная таблица по категории наличия детей и задолжности
print(data_pivot)

                       debt
children_category          
бездетная          0.075258
двудетная          0.094542
многодетная        0.085526
однодетная         0.092346


### Вывод

Бездетные семьи с большей вероятностью выплатят кредит. А многодетным семьям безопаснее выдавать кредит, чем (одно\двум)детным. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
data_pivot2 = data.pivot_table(index=['family_category'], values='debt')
print(data_pivot2)

                     debt
family_category          
в браке          0.075452
не в браке       0.088865


### Вывод

Люди, состоящие в браке, выплачивают кредит чаще на 1.3% чем не женатые люди

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
data_pivot3 = data.pivot_table(index=['total_income_category'], values='debt') #сводная таблица по категории дохода и задолжности
print(data_pivot3)

                           debt
total_income_category          
высокий                0.064103
выше среднего          0.080753
низкий                 0.061828
средний                0.083370


### Вывод

Люди с высоким и низким доходом выплачивают кредит в  чаще в среднем на 1.9% чем люди с доходом средним и выше среднего. Реже всего выплачивают кредит люди со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
data_pivot4 = data.pivot_table(index=['purpose_category'], values='debt') #сводная таблица по категории цели кредита и задолжности
print(data_pivot4)

                      debt
purpose_category          
автомобиль        0.093590
жилье             0.069058
недвижимость      0.074634
образование       0.092200
свадьба           0.080034


### Вывод

Кредит на жилье выплачиваетчя чаще чем остальные. Больше всего просрочек по кредитам на автомобили и образование.

### Шаг 4. Общий вывод

#Основываясь на выводах, сделаных выше, можно резюмировать, что безопаснее всего выдавать кредиты на недвижимость и жилье. Среди категорий людей, чаще возвращающих кредиты - семейные пары без детей с низким или высоким уровнем дохода.


Отвечая на поставленный вопрос "Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", я бы сказал, что количество детей и семейное положение влияет на то вернет ли заещик кредит в срок.

Люди, которые состоят в браке и не имеют детей - самые надежные заемщики. Они с большей долей вероятности вернут деньги в срок.

Люди, которые не состоят в браке и имеют 1-2 ребенка - самые ненадежные заемщики.

Также по результатам исследования можно сделать вывод, что кредиты, выданные на свадьбу или образование, имеют повышенный риск, в отличие от кредита на жилье.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.